In [1]:
import cbx as cbx
from cbx.dynamics.cbo import CBO
import numpy as np

import torch
import torch.nn as nn
import torchvision
from cbx.noise import anisotropic_noise
from torch.func import functional_call, stack_module_state, vmap

# Load data
We load the train and test data.

In [4]:
#%% load MNIST
data_path = "../../../datasets/"
transform = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST(data_path, train=True, transform=transform, download=False)
test_data = torchvision.datasets.MNIST(data_path, train=False, transform=transform, download=False)
train_loader = torch.utils.data.DataLoader(train_data, batch_size=64,shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=64,shuffle=False, num_workers=0)

# Load model

In [7]:
from models import Perceptron
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Perceptron().to(device)

In [48]:
import math
N = 50
models = [Perceptron() for _ in range(N)]
params, buffers = stack_module_state(models)
pnames = params.keys()
pshapes = {p: params[p][0,...].shape for p in pnames}
pcuts = []
for p in pshapes.items():
    a = 0
    if len(pcuts)>0:
        a = pcuts[-1]
    pcuts.append(a + math.prod(p))
def flatten_parameters(params, pnames):
    return torch.concatenate([params[pname].view(N,-1).detach() for pname in pnames], dim=-1)

x = flatten_parameters(params,pnames)

TypeError: can't multiply sequence by non-int of type 'torch.Size'

In [43]:
pshapes

{'linear.weight': torch.Size([10, 784]),
 'linear.bias': torch.Size([10]),
 'bn.weight': torch.Size([10]),
 'bn.bias': torch.Size([10])}

In [38]:
def eval_model(x, w, pshapes):
    w = torch.tensor(w, dtype=torch.float32)
    wcut = np.prod(wshape)
    params = {p: w[:wcut].view(pshapes[p]) for p in pshapes.keys()} 
    return functional_call(model, (params, buffers), x)

torch.Size([50, 7870])

In [27]:
model.linear.weight

Parameter containing:
tensor([[ 1.8989e-02,  7.2750e-03, -3.2954e-02,  ...,  3.1061e-02,
         -1.7993e-02, -1.1871e-02],
        [ 3.1170e-02, -2.1222e-03, -2.4572e-02,  ...,  3.1753e-02,
         -3.1188e-02, -4.8512e-03],
        [-2.7340e-03,  2.3394e-04,  1.0914e-02,  ..., -3.1603e-02,
          3.3151e-02,  2.7347e-02],
        ...,
        [-1.9654e-02,  3.3868e-02,  2.4875e-02,  ..., -3.2911e-02,
         -1.3968e-02, -5.1924e-04],
        [-1.5010e-02,  2.7146e-02, -5.8646e-03,  ...,  1.0248e-03,
          1.7836e-02, -1.7886e-02],
        [ 3.8642e-05, -1.6629e-02,  3.5446e-02,  ..., -3.0258e-02,
         -2.8792e-02,  3.4296e-02]], requires_grad=True)

In [ ]:
class objective:
    def __init__(self, train_loader, N, wshape, bshape):
        self.train_loader = train_loader
        self.data_iter = iter(train_loader)
        self.N = N
        self.wshape = wshape
        self.bshape = bshape
        self.wcut = np.prod(wshape)
        self.epochs = 0
        
    def __call__(self, w):
        (x,y) = next(self.data_iter, (None, None))
        if x is None:
            self.data_iter = iter(self.train_loader)
            (x,y) = next(self.data_iter)
            self.epochs += 1
        
        x, y = x.to(device), y.to(device)
        params = {'linear.weight': w[...,:self.wcut].view(self.N, *self.wshape),
                  'linear.bias': w[...,self.wcut:].view(self.N,*self.bshape)} 
            
        out = vmap(wrapper, (0, 0, None, None))(params, {}, x, y)
        return out